In [1]:
function node_master(node_pool)
    #master = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV)))
    master = Model(optimizer_with_attributes(() -> Cbc.Optimizer(Cbc_ENV)))
    set_optimizer_attribute(master, "OutputFlag", 0)
    
    # Define the variables α_mk
    # the first index is the machine index
    # the second index is the pattern index
    alpha = Array{Array{VariableRef,1},1}(undef,D.m)
    for m in 1:D.m
        alpha[m]=Array{VariableRef,1}(undef,size(node_pool[m],1))
    end
    for m in 1:D.m
        for k in 1:size(node_pool[m],1)
            alpha[m][k]=@variable(master,lower_bound=0)
            set_name(alpha[m][k], "alpha_$m,$k")
        end
    end
    
    # Constraint: Every job should be assigned to one machine  
    job = @constraint(master,[i in 1:D.n],sum( sum(node_pool[m][k][i+1] * alpha[m][k] for k in 1:size(node_pool[m],1)) for m in 1:D.m ) == 1)
    
    # Constraint: One pattern should be chosen for each machine
    machine = @constraint(master,[m in 1:D.m] sum(alpha[m][k] for k in 1:size(node_pool[m],1)) == 1)
    
    # Objective: Minimize the total cost of selected patterns
    @objective(master,Min,sum( sum( node_pool[m][k][1] * alpha[m][k] for m in 1:D.m) for k in 1:size(node_pool[m],1)) )

    # Return the constraint and variable references along with the model
    return job,machine,alpha,master
end

LoadError: UndefVarError: @variable not defined